In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:1


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [6]:
label = "KG_swish"
loss_thresh = 0.01

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [7]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [8]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Sigmoid()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xt_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [9]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [10]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [11]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*11)
  start_time = time.time() 
  thresh_flag = 0

  xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
  xt_coll = torch.from_numpy(xt_coll).float().to(device)
  xt_BC = torch.from_numpy(xt_BC).float().to(device)
  y_BC = torch.from_numpy(y_BC).float().to(device)

  f_hat = torch.zeros(xt_coll.shape[0],1).to(device)

  for i in range(max_iter):
    train_step(xt_BC, y_BC, xt_coll,f_hat,i)
    
    loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    

    

  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))

In [12]:

#for tune_reps in range(4,5):
max_reps = 10 #10
max_iter = 100 #100


train_loss_full = []
test_mse_full = []
test_re_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))
time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

N_I = 200  #Total number of data points for 'y'
N_B = 400
N_f = 10000 #Total number of collocation points

for reps in range(max_reps):
  print(reps)

  train_loss = []
  test_mse_loss = []
  test_re_loss = []
  beta_val = []

  torch.manual_seed(reps*36)

  layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

  PINN = Sequentialmodel(layers)

  PINN.to(device)

  'Neural Network Summary'
  print(PINN)

  params = list(PINN.parameters())


  optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.1, 
                            max_iter = 20, 
                            max_eval = 30, 
                            tolerance_grad = 1e-8, 
                            tolerance_change = 1e-8, 
                            history_size = 100, 
                            line_search_fn = 'strong_wolfe')




  nan_flag = train_model(max_iter,reps)

  torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
  train_loss_full.append(train_loss)
  test_mse_full.append(test_mse_loss)
  test_re_full.append(test_re_loss)
  #elapsed_time[reps] = time.time() - start_time
  beta_full.append(beta_val)

  #print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

0
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 58.234894 Test MSE 8.478368929871452 Test RE 1.3917595678250378
1 Train Loss 54.276566 Test MSE 7.921540478804159 Test RE 1.3452806117606328
2 Train Loss 49.209198 Test MSE 8.15753356495465 Test RE 1.3651723859154523
3 Train Loss 46.18009 Test MSE 8.466621798569216 Test RE 1.390795

97 Train Loss 8.478026 Test MSE 4.814268591871501 Test RE 1.0487530054311536
98 Train Loss 8.133154 Test MSE 4.8634199919197245 Test RE 1.0540930457275544
99 Train Loss 7.9841175 Test MSE 4.745120020371364 Test RE 1.0411940104463138
Training time: 155.79
1
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 56.48062 Test MSE 8.8112490

94 Train Loss 2.7643912 Test MSE 1.635398227150764 Test RE 0.6112513363214565
95 Train Loss 2.747778 Test MSE 1.6292913402850522 Test RE 0.6101090047466158
96 Train Loss 2.7293153 Test MSE 1.6498707382215096 Test RE 0.6139500234698606
97 Train Loss 2.6832957 Test MSE 1.6377979809568008 Test RE 0.6116996415056583
98 Train Loss 2.611024 Test MSE 1.628266487908211 Test RE 0.609917090014797
99 Train Loss 2.5940313 Test MSE 1.6406168469580524 Test RE 0.612225823013911
Training time: 160.20
2
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bia

91 Train Loss 0.1997531 Test MSE 0.03247856846326618 Test RE 0.08614032375683801
92 Train Loss 0.1534611 Test MSE 0.022556634341844798 Test RE 0.07178690174193834
93 Train Loss 0.13124685 Test MSE 0.01599851216618634 Test RE 0.0604571761581361
94 Train Loss 0.11760164 Test MSE 0.015128537280280794 Test RE 0.0587904155829883
95 Train Loss 0.105784185 Test MSE 0.011978696401927386 Test RE 0.05231338701960943
96 Train Loss 0.09945594 Test MSE 0.010521093950961687 Test RE 0.04902736016410579
97 Train Loss 0.08869704 Test MSE 0.009297338678613308 Test RE 0.04608794946616616
98 Train Loss 0.07918316 Test MSE 0.00877747198390474 Test RE 0.04478089675580298
99 Train Loss 0.06733979 Test MSE 0.0063587514097013615 Test RE 0.03811482970796047
Training time: 159.10
3
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in

87 Train Loss 17.154518 Test MSE 5.16319537136175 Test RE 1.0860938128168918
88 Train Loss 17.113878 Test MSE 5.07106116677499 Test RE 1.076359838330219
89 Train Loss 17.071491 Test MSE 4.997909761241757 Test RE 1.0685682489062367
90 Train Loss 17.039461 Test MSE 4.978930289337091 Test RE 1.0665373847325816
91 Train Loss 17.01722 Test MSE 4.98789965167234 Test RE 1.0674976166795709
92 Train Loss 17.000982 Test MSE 5.03965869050138 Test RE 1.0730219911922012
93 Train Loss 16.888487 Test MSE 4.954229215373788 Test RE 1.0638884849384425
94 Train Loss 16.622673 Test MSE 4.846536825508617 Test RE 1.0522618343479384
95 Train Loss 16.358397 Test MSE 4.760368002790082 Test RE 1.0428655566101372
96 Train Loss 16.202103 Test MSE 4.656171492477265 Test RE 1.0313891158944928
97 Train Loss 15.847779 Test MSE 4.995391757433824 Test RE 1.068299036572494
98 Train Loss 15.665281 Test MSE 4.875972293151471 Test RE 1.0554524560902263
99 Train Loss 15.347095 Test MSE 4.774045012642242 Test RE 1.0443626102

...

80 Train Loss 0.76321316 Test MSE 0.679659720151431 Test RE 0.3940522634485103
81 Train Loss 0.7482951 Test MSE 0.6715270143630702 Test RE 0.3916875833073536
82 Train Loss 0.73844403 Test MSE 0.6665991054678596 Test RE 0.390247764159478
83 Train Loss 0.7206225 Test MSE 0.6627383763344 Test RE 0.3891160279060417
84 Train Loss 0.7083092 Test MSE 0.665464303482562 Test RE 0.3899154485514802
85 Train Loss 0.70121807 Test MSE 0.6735872640262426 Test RE 0.39228797330755427
86 Train Loss 0.6942973 Test MSE 0.6766666857087209 Test RE 0.39318365723268983
87 Train Loss 0.6737659 Test MSE 0.6790144372648901 Test RE 0.3938651583747126
88 Train Loss 0.6530762 Test MSE 0.6750215951659586 Test RE 0.392705418579839
89 Train Loss 0.6451451 Test MSE 0.6649131972779259 Test RE 0.3897539603021028
90 Train Loss 0.6370374 Test MSE 0.662319429230758 Test RE 0.38899301948388115
91 Train Loss 0.6263292 Test MSE 0.6551148816467254 Test RE 0.3868715493843229
92 Train Loss 0.6125631 Test MSE 0.641880069135171 Tes

74 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
75 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
76 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
77 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
78 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
79 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
80 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
81 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
82 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
83 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
84 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
85 Train Loss 0.16030855 Test MSE 0.010532261004856207 Test RE 0.04905337200168333
86 T

69 Train Loss 18.663733 Test MSE 2.2334206176768174 Test RE 0.7143208401784878
70 Train Loss 18.616432 Test MSE 2.233292932332112 Test RE 0.7143004209154024
71 Train Loss 18.550144 Test MSE 2.2265490732561193 Test RE 0.7132211215140832
72 Train Loss 18.48906 Test MSE 2.2082690731591343 Test RE 0.7102873095820581
73 Train Loss 18.418724 Test MSE 2.188184016484424 Test RE 0.7070497629297676
74 Train Loss 18.326185 Test MSE 2.122472388171591 Test RE 0.6963524129650502
75 Train Loss 18.279594 Test MSE 2.120336526126933 Test RE 0.6960019520923695
76 Train Loss 18.203716 Test MSE 2.088943153379544 Test RE 0.6908302896864723
77 Train Loss 18.123842 Test MSE 2.024733536804415 Test RE 0.6801301046365397
78 Train Loss 17.64127 Test MSE 1.974631356242477 Test RE 0.6716624585991317
79 Train Loss 16.748081 Test MSE 1.8834583187833382 Test RE 0.6559731556032077
80 Train Loss 16.135677 Test MSE 1.7873449070700507 Test RE 0.6390167525910719
81 Train Loss 15.785927 Test MSE 1.7375955846942561 Test RE 0

66 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
67 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
68 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
69 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
70 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
71 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
72 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
73 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
74 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
75 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
76 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
77 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035
78 Train Loss 57.017754 Test MSE 8.725380877053066 Test RE 1.411888032738035

60 Train Loss 0.010204628 Test MSE 0.0007903034307075786 Test RE 0.013437082817635579
61 Train Loss 0.009859505 Test MSE 0.0008624144401494265 Test RE 0.014036734004078785
62 Train Loss 0.009669168 Test MSE 0.0009196877606715471 Test RE 0.014495335232985304
63 Train Loss 0.009252343 Test MSE 0.0009213172351867989 Test RE 0.014508170744894902
64 Train Loss 0.00894065 Test MSE 0.0010256352433077008 Test RE 0.015307509167417646
65 Train Loss 0.008824288 Test MSE 0.001047139852899565 Test RE 0.01546715382047078
66 Train Loss 0.008608729 Test MSE 0.0010400202953179984 Test RE 0.015414483153964048
67 Train Loss 0.00794317 Test MSE 0.001133604629763286 Test RE 0.016093068718478556
68 Train Loss 0.007626447 Test MSE 0.001087292873381892 Test RE 0.01576091152201421
69 Train Loss 0.007327945 Test MSE 0.001081776773318168 Test RE 0.015720881226071938
70 Train Loss 0.0070026545 Test MSE 0.0010978308395598454 Test RE 0.015837104177569616
71 Train Loss 0.006832489 Test MSE 0.001054082301732122 Test 

In [13]:
import scipy.io as sio
import numpy as np

In [14]:
for tune_reps in range(4,5):
    label = "KG_swish_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
 
    #re = np.array(data["test_re_loss"])
    #print("tune_reps"," ",np.mean(re[0][0]))

FileNotFoundError: [Errno 2] No such file or directory: 'KG_swish_tune4.mat'

In [ ]:
tune_reps = 4
label = "KG_swish_tune"+str(tune_reps)+".mat"
data = sio.loadmat(label)
for k in range(10):
    print(tune_reps," ",data["test_re_loss"][0][k].shape)

In [ ]:
np.mean(data["test_re_loss"][:,-1])

In [15]:
a = 0
for i in range(10):
    a = a+ test_re_full[i][-1]
    print(test_re_full[i][-1])
    
print("a = ",a/10)

1.0411940104463138
0.612225823013911
0.03811482970796047
1.0443626102221957
1.4076454383354298
0.3801525612974972
0.04905337200168333
0.14675938294395968
1.411888032738035
0.011896791757174185
a =  0.6143292852464161
